<a href="https://colab.research.google.com/github/joaco2v/ProyectoDS-JoaquinVicuna/blob/main/ProyectoDSParteI_JoaquinVicuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto DS Parte I
## Contexto:
## Hipotesis:


In [ ]:
import pandas as pd
from datetime import datetime, time
import pytz
!pip install workalendar
from workalendar.america import Chile

#Generamos variable con registros de sismos < 4.5
url_df_github = "https://raw.githubusercontent.com/joaco2v/ProyectoDS-JoaquinVicuna/refs/heads/main/sismos_chile.csv"
#Cargamos csv
df = pd.read_csv(url_df_github)
#Se fija fecha en zona horaria UTC
df["Fecha UTC"] = pd.to_datetime(df["time"],utc=True)
#Generamos una columna con la zona horaria chilena
santiago_tz = pytz.timezone("America/Santiago")
df["Fecha UTC CL"] = df["Fecha UTC"].dt.tz_convert(santiago_tz)

#Se determinará si el sismo fue en un horario habil o inhabil

cal = Chile()

#Se define funcion con los horarios habiles e inhabiles

def es_habil(fecha_hora):
    fecha = fecha_hora.date()
    hora = fecha_hora.time()
    dia_semana = fecha_hora.weekday() #0 = Lunes, 6 = Domingo

    #Verificamos si es fin de semana es decir si es Sabado o Domingo

    if dia_semana >=5:
        return "No habil"

    #Verificamos si es dia feriado

    if cal.is_holiday(fecha):
        return "No habil"

    #Verificamos horario laboral definido (7:30 AM a 7:30 PM)
    if time(7,30) <= hora <= time(19,30):
        return "Habil"
    else:
        return "No habil"

df["Horario_Habil"] = df["Fecha UTC CL"].apply(es_habil)

#Formateamos las columnas finales

df["Fecha UTC CL"]= df["Fecha UTC CL"].dt.strftime("%Y-%m-%d %H:%M:%S")

#Ahora definiremos si es terremoto según la semantica chilena,
# es decir ajustaremos cualquier sismo >= 6.5 como terremoto, y si es < 6.5 será considerado temblor
def es_terremoto(mag):
    if mag < 6.5:
        return False
    else:
        return True
df["Terremoto"] = df["mag"].apply(es_terremoto)

#Luego, guardamos el resultado:
df.to_csv("sismos_chile_actualizado.csv",index=False)
